In [4]:
!pip install TensorboardX

     |████████████████████████████████| 204kB 7.4MB/s eta 0:00:01


In [1]:
import numpy as np
import pypianoroll
from pypianoroll import Multitrack, Track
from matplotlib import pyplot as plt
import pretty_midi
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
import pygame
import os
import pickle
import subprocess

%matplotlib inline

import matplotlib

import seaborn as sns


from IPython import display

from torch.utils.data import DataLoader
import torch
from torch.optim import Adam
from torch.autograd import Variable

import torch.nn as nn
import pandas as pd
from skimage import io, transform
import tensorflow as tf
import logging
from tensorflow import layers


import os
import zipfile

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Parse a MIDI file to a `pypianoroll.Multitrack` instance

multitrack = Multitrack('./data/rock.mid')
for idx, track in enumerate(multitrack.tracks):
    pianoroll = track.pianoroll


#print (multitrack.beat_resolution,)
num_track = len(multitrack.tracks)
downbeats = multitrack.get_downbeat_steps()
#print (downbeats,num_track)

In [3]:
pianoroll.shape

(5664, 128)

In [4]:

import os
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import Activation, UpSampling2D, Conv2D
from keras.layers.merge import _Merge
from keras.layers.convolutional import Convolution2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.callbacks import TensorBoard
from keras.utils.np_utils import to_categorical
from keras import backend as K
from functools import partial
import tensorflow as tf

import sys

Using TensorFlow backend.


In [5]:
def make_generator():
    D = 64 # model size

    model = Sequential()
    model.add(Dense(256 * D, input_dim=100))
    model.add(Reshape((4, 4, 16 * D)))
    model.add(Activation('relu'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(8 * D, (5, 5), padding='same'))
    #model.add(Conv2DTranspose(8*D, (5, 5), strides=(2,2), padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(4 * D, (5, 5), padding='same'))
    #model.add(Conv2DTranspose(4*D, (5, 5), strides=(2,2), padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(2 * D, (5, 5), padding='same'))
    #model.add(Conv2DTranspose(2*D, (5, 5), strides=(2,2), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(D, (5, 5), padding='same'))
    model.add(UpSampling2D(size=(2, 2)))
    #model.add(Conv2DTranspose(D, (5, 5), strides=(2,2), padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(1, (5, 5), padding='same'))
    #model.add(Conv2DTranspose(1, (5, 5), strides=(2,2), padding='same'))

    model.add(Activation('tanh'))

    return model

def make_discriminator(nb_categories):
    D = 64 # model size

    input_data = Input(shape=(128, 128, 1))
    x = Conv2D(D, (5, 5), strides=(2,2), padding='same')(input_data)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(D * 2, (5, 5), strides=(2,2), kernel_initializer='he_normal',padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(D * 4, (5, 5), strides=(2,2), kernel_initializer='he_normal',padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(D * 8, (5, 5), strides=(2,2), kernel_initializer='he_normal',padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(D * 16, (5, 5), strides=(2,2), kernel_initializer='he_normal', padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)
    real_fake = Dense(1, kernel_initializer='he_normal', name='real_fake')(x) # no activation for wasserstein_loss
    categories = Dense(nb_categories, kernel_initializer='he_normal', name='categories', activation='softmax')(x)

    model = Model(input_data, [real_fake, categories])

    return model

generator = make_generator()
generator

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
